# ANP Forecasting de Preços de Combustiveis

### Carregamento de Dados

In [1]:
# Carregamento de bibliotecas
import pandas as pd

In [4]:
# Carregamento de dados
url_data = "https://raw.githubusercontent.com/AurelioGuilherme/ANP-ETL/refs/heads/main/Data/Data-silver/dados_combustiveis_anp.csv"
df = pd.read_csv(url_data)
df.head()

,DATA INICIAL,DATA FINAL,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,COEF DE VARIAÇÃO REVENDA,periodo_referencia
0,2023-08-27 00:00:00,2023-09-02 00:00:00,SERGIPE,ARACAJU,ETANOL HIDRATADO,16,R$/l,4.79,0.095708,4.57,4.89,0.020,2023-08-27 a 2023-09-02
1,2023-08-27 00:00:00,2023-09-02 00:00:00,PARA,BELEM,ETANOL HIDRATADO,7,R$/l,4.66,0.328655,4.29,5.19,0.071,2023-08-27 a 2023-09-02
2,2023-08-27 00:00:00,2023-09-02 00:00:00,MINAS GERAIS,BELO HORIZONTE,ETANOL HIDRATADO,40,R$/l,3.50,0.108223,3.34,3.79,0.031,2023-08-27 a 2023-09-02
3,2023-08-27 00:00:00,2023-09-02 00:00:00,RORAIMA,BOA VISTA,ETANOL HIDRATADO,4,R$/l,4.94,0.140000,4.87,5.15,0.028,2023-08-27 a 2023-09-02
4,2023-08-27 00:00:00,2023-09-02 00:00:00,DISTRITO FEDERAL,BRASILIA,ETANOL HIDRATADO,49,R$/l,3.77,0.142499,3.58,3.99,0.038,2023-08-27 a 2023-09-02


### Tratamento de Dados

### Modelagem

### Validação